<a href="https://colab.research.google.com/github/mromaoro/buscador-dados-googlemaps/blob/main/Buscador_Google_Maps_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googlemaps pandas -q
print("Bibliotecas instaladas.")

import pandas as pd
import googlemaps
import time
from google.colab import userdata

try:
    api_key = userdata.get('GOOGLE_MAPS_API_KEY')
    gmaps = googlemaps.Client(key=api_key)
    print("Chave de API carregada com sucesso.")
except Exception as e:
    print(f"Erro ao carregar a chave de API. Verifique se ela foi salva como 'GOOGLE_MAPS_API_KEY' nos segredos do Colab. Detalhes: {e}")
    gmaps = None

def get_region(lat, lon):
    """
    Categoriza um ponto (lat, lon) em uma região predefinida.
    """
    regioes = {
        'Vila_Sao_Jorge': {
            'lat_min': -14.182321,  # Ponto Sul
            'lat_max': -14.170171,  # Ponto Norte
            'lon_min': -47.825727,  # Ponto Oeste
            'lon_max': -47.803926   # Ponto Leste
        },
        'Alto_Paraiso_de_Goias': {
            'lat_min': -14.159599,  # Ponto Sul
            'lat_max': -14.115395,  # Ponto Norte
            'lon_min': -47.538649,  # Ponto Oeste
            'lon_max': -47.495980   # Ponto Leste
        },
        'Cavalcante': {
            # Mantendo as coordenadas anteriores, caso você queira refazê-las
            'lat_min': -13.784, 'lat_max': -13.750,
            'lon_min': -47.464, 'lon_max': -47.445
        }
    }

    for nome_regiao, coords in regioes.items():
        if (coords['lat_min'] <= lat <= coords['lat_max']) and \
           (coords['lon_min'] <= lon <= coords['lon_max']):
            return nome_regiao

    return 'Outro'

def get_all_places(queries, location="Alto Paraíso de Goiás"):
    """
    Executa múltiplas buscas para obter uma lista completa e remove duplicatas.
    """
    if not gmaps:
        return pd.DataFrame()

    all_places = {} # Usaremos um dicionário para remover duplicatas facilmente

    for query in queries:
        full_query = f"{query} em {location}"
        response = gmaps.places(query=full_query)

        # Coleta os resultados da primeira página
        results = response.get('results', [])

        # Coleta os resultados de páginas adicionais (se houver)
        while response.get('next_page_token'):
            time.sleep(2)  # Atraso necessário entre as requisições de página
            response = gmaps.places(query=full_query, page_token=response['next_page_token'])
            results.extend(response.get('results', []))

        for place in results:
            place_id = place.get('place_id')
            if place_id and place_id not in all_places:
                try:
                    all_places[place_id] = {
                        'nome': place.get('name'),
                        'endereco': place.get('formatted_address'),
                        'latitude': place['geometry']['location']['lat'],
                        'longitude': place['geometry']['location']['lng'],
                        'rating': place.get('rating', 'N/A'),
                        'total_avaliacoes': place.get('user_ratings_total', 'N/A'),
                        'nivel_preco': place.get('price_level', 'N/A'),
                        'categorias': place.get('types', 'N/A'),
                        'regiao': get_region(place['geometry']['location']['lat'], place['geometry']['location']['lng']),
                        'termo_da_pesquisa': query
                    }
                except KeyError:
                    continue

        print(f"Busca por '{query}' concluída. Total de locais únicos: {len(all_places)}")
        time.sleep(2) # Pausa entre as buscas para evitar erros

    return pd.DataFrame(list(all_places.values()))

# --- Execução do Script ---
if gmaps:
    # Lista de buscas que você quer realizar. Você pode adicionar mais itens aqui.
    searches = [
        "pousadas",
        "hoteis",
        "hospedagem",
        "bistrô",
        "restaurantes",
        "agências de turismo",
        "lojas de artesanato",
        "comércios",
        "cafeterias",
        "centros holísticos",
        "lojas de produtos naturais",
        "supermercados",
        "pousada em são jorge",
        "hospedagem em são jorge",
        "airbnb em alto paraiso de goias",
        "airbnb em são jorge",
        "chalé em alto paraiso de goias",
        "bangalo em alto paraiso de goias",
        "Bistro em são jorge",
        "restaurante em são jorge",
        "pizzaria em são jorge",
        "churrascaria em são jorge",
        "risoteria santo cerrado",
        "Mirante",
        "Bar",
        "Hostel",
        "atração turística",
        "retiro",
        "hamburgueria"
    ]

    df_completo = get_all_places(searches, location="Alto Paraíso de Goiás")

    if not df_completo.empty:
        # Exibe o DataFrame
        print("\n--- Dados Consolidados ---")
        print(df_completo.head(10))
        print(f"\nTotal de locais únicos coletados: {len(df_completo)}")

        # Salva o DataFrame em um arquivo CSV
        filename = f'comércios_alto_paraíso.csv'
        df_completo.to_csv(filename, index=False)
        print(f"\nArquivo '{filename}' criado com sucesso!")
    else:
        print("Não foi possível coletar dados. Verifique a chave de API e as consultas.")

In [ ]:
# Busca de teste
if gmaps:
    # A busca mais específica possível
    response = gmaps.places(query="Guianna Hostel São Jorge Chapada dos Veadeiros")

    if response.get('results'):
        primeiro_resultado = response['results'][0]
        print("\nO Guianna Hostel foi encontrado!")
        print(f"Nome: {primeiro_resultado.get('name')}")
        print(f"ID: {primeiro_resultado.get('place_id')}")
        print("-" * 20)
        # Você pode imprimir o resultado completo para ver todos os campos disponíveis
        # print(primeiro_resultado)
    else:
        print("\nO Guianna Hostel não foi encontrado com esta busca específica.")